In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
import random
import numpy as np
from tqdm.notebook import tqdm
from tabulate import tabulate
from omnibelt import load_json, save_json
from pathlib import Path
from collections import Counter
import requests
import json
from googlefinance import getQuotes
from bs4 import BeautifulSoup
import requests, sys, lxml.html
# from finquant.portfolio import build_portfolio

# import yfinance as yf
from yahooquery import Ticker

# from src import URLS, STRATEGY, extract_html, extract_table, load_etf_snapshot, \
#     load_etf_analyst, load_etf_pie, load_etf_holdings, get_yf, get_ticker, \
#     get_recommendations, get_fund_holdings, get_fund_sectors, get_type, get_fund_categories, \
#     load_stock_snapshot, load_stock_contained, load_stock_history, dicts_to_table, SITES, \
#     clear_loaded, load, TYPES, collect

from src import *

In [2]:
names = []
ratios = {}
for group, tickers in STRATEGY.items():
    ratios.update({k:v/100 for k,v in tickers.items()})

names = list(k for k,v in ratios.items() if v > 0)

cats = TYPES.copy()
# cats = {}
# for n in names:
#     t = get_type(n)
#     if t not in cats:
#         cats[t] = []
#     cats[t].append(n)

name = random.choice(names)
name

'SKYY'

In [3]:
sum(ratios.values())

1.0000000000000004

In [39]:
cat = 'ETF'
# cat = 'EQUITY'
group = 'snapshot'
group = 'holdings'

In [40]:
# for site in SITES[cat]:
#     for ticker in cats[cat]:
#         print(URLS[cat][site].format(ticker=ticker))
#     break

In [41]:
full = load(cat, group, pbar=tqdm)
len(full), full.keys()

  0%|          | 0/40 [00:00<?, ?it/s]

(40,
 dict_keys(['VONG', 'IBB', 'CIBR', 'FHLC', 'LIT', 'FIDU', 'FNCL', 'FMAT', 'ICLN', 'PHO', 'ACES', 'SPY', 'KWEB', 'PBW', 'ASHR', 'CNYA', 'MCHI', 'EWG', 'FLJP', 'EWY', 'EWC', 'VGK', 'IWV', 'SPDW', 'ICVT', 'SCHP', 'EMXC', 'IGOV', 'IAU', 'IXUS', 'LQD', 'AGG', 'IAGG', 'SOCL', 'VXF', 'VTI', 'ESPO', 'KOMP', 'SKYY', 'XITK']))

In [45]:
rtk, raw = next(iter(full.items()))
rtk, raw.keys() if isinstance(raw, dict) else len(raw)

('VONG', 405)

In [44]:
raw

[['AAPL', 'Apple Inc', 'United States', 0.1094],
 ['MSFT', 'Microsoft Corp', 'United States', 0.0913],
 ['AMZN', 'Amazon.com Inc', 'United States', 0.0765],
 ['FB', 'Facebook Inc A', 'United States', 0.0379],
 ['TSLA', 'Tesla Inc', 'United States', 0.0239],
 ['GOOGL', 'Alphabet Inc A', 'United States', 0.0233],
 ['GOOG', 'Alphabet Inc Class C', 'United States', 0.0231],
 ['V', 'Visa Inc Class A', 'United States', 0.0202],
 ['NVDA', 'NVIDIA Corp', 'United States', 0.018000000000000002],
 ['MA', 'Mastercard Inc A', 'United States', 0.0169],
 ['UNH', 'UnitedHealth Group Inc', 'United States', 0.0151],
 ['PYPL', 'PayPal Holdings Inc', 'United States', 0.0143],
 ['ADBE', 'Adobe Inc', 'United States', 0.0131],
 ['NFLX', 'Netflix Inc', 'United States', 0.011899999999999999],
 ['CRM', 'Salesforce.com Inc', 'United States', 0.011200000000000002],
 ['MRK', 'Merck & Co Inc', 'United States', 0.0101],
 ['ABBV', 'AbbVie Inc', 'United States', 0.009899999999999999],
 ['QCOM', 'Qualcomm Inc', 'United

In [30]:
data = {}

In [31]:
features = {
    # EQUITY
    
    'Employees': ('company', 'employees'),
#     'Country': ('company', 'country'),
#     'Sector': ('company', 'sector'),
#     'Industry': ('company', 'industry'),
    
    'Performance': ('basics', 'Price Performance (Last 52 Weeks)'),
    'Score': ('analyst', 'score'),
    'Verdict': ('analyst', 'verdict'),
    
    
    # ETF
    
#     'P/E': ('stats', 'Price / Earnings (Trailing Twelve Months)'),
#     'Yield': ('stats', 'Distribution Yield (TTM)'),
#     'Assets': ('info', 'Net Assets'),
#     'Expense': ('info', 'Net Expense Ratio'),
#     'Date': ('info', 'Inception Date'),
    
#     'SEC Yield': ('stats', '30-day SEC Yield date'),
#     'ratings': 'ratings',
#     'analyst': 'analyst',
    
}
cols = {
    'FactSet':int, 'Morningstar, Inc.':int, 
    'Tradability':int, 'Fit':int, 'Efficiency':int,
    'Employees': int, 'Country': str,
    'P/E': float, 'Yield': float,
    'Assets': float, 'Expense': float, 'Date': str,
    'Score': float,
}

In [32]:
data = collect(full, features, data=data)
tk, info = next(iter(data.items()))
len(data), tk, info.keys()

(20, 'TSLA', dict_keys(['Employees', 'Performance', 'Score', 'Verdict']))

In [33]:
# for k,v in data.items():
#     if 'Assets' in v and v['Assets'] is not None:
#         v['Assets'] /= 1e6 

In [34]:
info

{'Employees': 48016, 'Performance': 729.68, 'Score': 8.5, 'Verdict': 'Bullish'}

In [35]:
tbl = dicts_to_table(data, typs=cols, sort_by='Score', reverse=True)#, default=0)
# len(tbl)
print(tabulate(tbl[1:], headers=tbl[0]))

Ticker    Verdict         Score    Employees    Performance
--------  ------------  -------  -----------  -------------
SNE       Very Bullish      9.8       111700          51.45
TSLA      Bullish           8.5        48016         729.68
PDD       Bullish           8.3         5828         357.81
FB        Bullish           8.2        44942          29.86
BIDU      Bullish           7.9        37779          59.94
MSFT      Very Bullish      7.8       163000          37.37
COST      Bullish           7.8       273000          32.17
NVDA      Bullish           7.6        13775         127.13
PYPL      Neutral           7.1        23200         115.99
AAPL      Bullish           7         147000          76.19
SHOP      Neutral           6.7         5000         176.72
AMZN      Neutral           6.5       798000          71.66
AMD       Neutral           6.3        11400          90.88
GOOGL     Neutral           5.2       118899          27.8
NFLX      Neutral           4.6         8

In [24]:
# cols = tbl[0][1:]
# vals = [row[1:] for row in tbl[1:]]
# tickers = [row[0] for row in tbl[1:]]
# # print(len(vals))
# dat = np.array(vals).astype(float)
# dat[(dat-dat)!=0] = 0
# # print(dat.shape)
# plot_parallel_coords(dat, dim_names=cols, categories=tickers, include_legend=None)
# pass